In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Spice.Netlist import Circuit
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Unit import *
from PySpice.Probe.Plot import plot
from PySpice.Logging.Logging import setup_logging

setup_logging()

libraries_path = find_libraries()
spice_library = SpiceLibrary(libraries_path)
spice_library = SpiceLibrary(find_libraries())

In [2]:
rsh_top_values = np.linspace(0, 500, 1000)  # Range of rs_top values
v_oc_values = []  # List to store v_oc values
i_sc_values = []  # List to store i_sc values
fill_factor_values = []  # List to store fill factor values
efficiency_values = []  # List to store efficiency values


for rsh_top in rsh_top_values:
    circuit = Circuit('One perovskite/silicon tandem cell')
    circuit.model('D1_top', 'D', IS = 8.8e-23, N =1)
    circuit.model('D2_top', 'D', IS = 2.62e-13, N =2)
    circuit.model('D1_bot', 'D', IS=1.3339e-14, N =1)
    circuit.model('D2_bot', 'D', IS=6.5674e-19, N =2)

    circuit.V('input', '6', circuit.gnd, 0@u_V)
    circuit.R('s_top', '5', '6', 0@u_Ohm)
    circuit.D('1_top', '5', '4', model='D1_top')
    circuit.D('2_top', '5', '4', model='D2_top')
    circuit.R('sh_top', '5', '4', rsh_top@u_Ohm)
    circuit.I('ph_top', '4', '5', 19.65@u_mA)
    circuit.R('c_top', '4', '3', 0@u_Ohm)
    circuit.R('c_bot', '3', '2', 0@u_Ohm)
    circuit.R('s_bot', '2', '1', 0@u_Ohm)
    circuit.D('1_bot', '1', circuit.gnd, model='D1_bot')
    circuit.D('2_bot', '1', circuit.gnd, model='D2_bot')
    circuit.R('sh_bot', '1', circuit.gnd, 12@u_kOhm)
    circuit.I('ph_bot', circuit.gnd, '1', 19.65@u_mA)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis = simulator.dc(Vinput=slice(0, 2, 0.0001))

    # Calculate open circuit voltage
    voltage = analysis.nodes['6'].as_ndarray()
    current = analysis.branches['vinput'].as_ndarray()
    v_oc_index = np.argmin(np.abs(current))
    v_oc = voltage[v_oc_index]
    v_oc_values.append(v_oc)

    # Calculate short circuit current
    i_sc_index = np.argmin(np.abs(voltage))
    i_sc = current[i_sc_index]
    i_sc_values.append(i_sc)
    
    
    # Calculate maximum power point
    power = voltage * current
    mpp_index = np.argmax(power)
    v_mpp = voltage[mpp_index]
    i_mpp = current[mpp_index]
    max_power = power[mpp_index]
    efficiency = (max_power/(0.1))*100
    efficiency_values.append(efficiency)
    
    # Calculate fill factor
    fill_factor = ((v_mpp * i_mpp) / (v_oc * i_sc))*100
    fill_factor_values.append(fill_factor)

2023-08-04 18:09:53,921 - PySpice.Spice.NgSpice.Shared.NgSpiceShared - Shared.WARNING - Unsupported Ngspice version 38


In [3]:
# 최소 fill factor 값을 찾고 해당 값을 가지는 저항 값을 구함
min_fill_factor = min(fill_factor_values)
min_fill_factor_index = fill_factor_values.index(min_fill_factor)
min_fill_factor_resistance = rsh_top_values[min_fill_factor_index]

print("Resistace value at minimum fill factor:", min_fill_factor_resistance, "Ohm")

Resistace value at minimum fill factor: 58.058058058058066 Ohm
